In [ ]:
import avapi
import avstack
import pandas

import os
from tqdm import tqdm

## Compute Results

In [ ]:
models = ['FasterRCNN', 'PointPillars']
train_apps = ['Ego', 'Infrastructure']
test_apps = ['Ego', 'Infrastructure']

data_dir_ego = '/data/spencer/CARLA/multi-sensor-v1/'
data_dir_infra = '/data/spencer/CARLA/multi-agent-v1/'

SM_ego = avapi.carla.CarlaScenesManager(data_dir_ego)
SM_infra = avapi.carla.CarlaScenesManager(data_dir_infra)

In [ ]:
dets[0].score

In [ ]:
out_dir = 'application_results'
n_scenes = 1
n_frames = 10
for model in models:
    # -- model-specific information
    if model.lower() == 'fasterrcnn':
        data_type = 'camera'
        metric = '2D_IoU'
        MMModel = avstack.modules.perception.object2dfv.MMDetObjectDetector2D
    elif model.lower() == 'pointpillars':
        data_type = 'lidar'
        MMModel = avstack.modules.perception.object3d.MMDetObjectDetector3D
        metric = '3D_IoU'
    else:
        raise NotImplementedError(model.lower())

    # -- loops
    for train_app in applications:
        if train_app.lower() == 'ego':
            dataset = 'carla'
        else:
            dataset = 'carla-infrastructure'
        M = MMModel(model=model.lower(), dataset=dataset)
        for test_app in applications:
            out_folder = os.path.join(out_dir, train_app, test_app)
            gt_dir = os.path.join(out_folder, 'ground-truth')
            det_dir = os.path.join(out_folder, 'detection-results')
            os.makedirs(gt_dir, exist_ok=True)
            os.makedirs(det_dir, exist_ok=True)
            
            ground_truth = {'boxes':[], 'labels':[], 'scores':[]}
            result_dict = {'boxes':[], 'labels':[], 'scores':[]}
            if test_app.lower() == 'ego':
                SM = SM_ego
            else:
                SM = SM_infra

            # -- loop frames
            for i_scene, scene in enumerate(SM.splits_scenes['val'][:n_scenes]):
                SD = SM.get_scene_dataset_by_name(scene)
                for frame in tqdm(SD.frames[:n_frames]):
                    # -- run perception
                    try:
                        if data_type == 'camera':
                            sensor = 'main_camera'
                            data = SD.get_image(frame=frame, sensor=sensor)
                        else:
                            sensor = 'main_lidar'
                            data = SD.get_lidar(frame=frame, sensor=sensor)
                        objs = SD.get_objects(frame=frame, sensor=sensor)
                    except FileNotFoundError, KeyError:
                        continue
                    dets = M(data=data)

                    # -- store outcomes
                    img_name = f'image_{i_scene:03d}-{frame:03d}.txt'
                    gt_lines = [[obj.obj_type, *map(str, list(obj.box.project_to_2d_bbox(data.calibration).box2d))] for obj in objs]
                    gt_string = '\n'.join([' '.join(line) for line in gt_lines])
                    with open(os.path.join(gt_dir, img_name), 'w') as f:
                        f.write(gt_string)
                    det_lines = [[det.obj_type, str(det.score), *map(str, det.box.box2d)] for det in dets]
                    det_string = '\n'.join([' '.join(line) for line in det_lines])
                    with open(os.path.join(det_dir, img_name), 'w') as f:
                        f.write(det_string)

In [ ]:
results = {'model':[], 'data_type':[], 'train_app':[], 'test_app':[], 'mAP':[]}

# -- calculate results
mAP_results = calculate_map_range(ground_truth, result_dict, 0.05, 0.95, 0.05)

# -- store results
results['model'].append(model)
results['data_type'].append(data_type)
results['train_app'].append(train_app)
results['test_app'].append(test_app)
results['mAP'].append(mAP_results

## Visualize Results

In [ ]:
df = pandas.DataFrame()

In [ ]:
subrow_formatter = '\tworowsubtablecenter{{{}}}{{\tworowsubtablecenter{{{}}}{{{}}}}}'